In [1]:
import pandas as pd
from pylab import *

FILE = "C:/Users/jtso/My Documents/BRCA.txt"

maf = pd.read_table(FILE, skiprows = 1, usecols=["Hugo_Symbol", "Chromosome", "Start_Position", "End_Position", "Tumor_Sample_Barcode"])
maf.head()

,Hugo_Symbol,Chromosome,Start_Position,End_Position,Tumor_Sample_Barcode
0,A1CF,10,52587953,52587953,TCGA-D8-A1J8-01A-11D-A13L-09
1,A1CF,10,52595854,52595854,TCGA-BH-A0HP-01A-12D-A099-09
2,A1CF,10,52595854,52595854,TCGA-BH-A0HP-01A-12D-A099-09
3,A1CF,10,52595937,52595937,TCGA-BH-A18P-01A-11D-A12B-09
4,A1CF,10,52596055,52596055,TCGA-AC-A2FB-01A-11D-A17D-09


In [2]:
print len(maf)

90490


In [3]:
FILE = "../hw6/hsa.gff3"
TABLE2 = "C:/Users/jtso/Downloads/targetScanS.gz"  # miRNA binding sites
TABLE3 = "C:/Users/jtso/Downloads/wgEncodeGencodePolyaV19 (1).gz"  # polyA sites
df = pd.read_table(FILE, comment="#", usecols=["Chromosome", "Start", "End", "Type", "miR Name"])

chr2df = {}
for chr in [str(i) for i in range(1,23)] + ["X", "Y"]:
    chr2df[chr] = df[df["Chromosome"] == "chr" + chr]
    
def pos2miR(chr, start, end):
    chunk = chr2df[str(chr)]  # shrink the search space to the given chr
    out = []
    for idx, row in chunk.iterrows():
        if row["Start"] <= start <= row["End"] or \
           row["Start"] <=  end  <= row["End"] or \
           (start < row["Start"] and row["End"] < end):
            # get the name (either hsa-mir-### or hsa-miR-###)
            miR = str(row["Type"]) + ";" + str(row["Start"]) + ";" + str(row["End"]) + ";" + row["miR Name"].split("Name=")[1].split(";")[0]
            out.append(miR)
    return out

c = pd.DataFrame()
ms = pd.DataFrame()
me = pd.DataFrame()
rs = pd.DataFrame()
re = pd.DataFrame()
t = pd.DataFrame()
i = pd.DataFrame()
s = pd.DataFrame()
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    try:
        hits = pos2miR(row["Chromosome"], (row["Start_Position"]), (row["End_Position"]))
    except:
        print row["Chromosome"]
        continue
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_Position"], row["End_Position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_Position"]])
        me = me.append([row["End_Position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

ATF5 19 50436338 50436338 TCGA-E2-A15K-06A-11D-A12Q-09 hits on miRNA_primary_transcript;50436321;50436394;hsa-mir-4751
ATF5 19 50436338 50436338 TCGA-E2-A15K-06A-11D-A12Q-09 hits on miRNA;50436330;50436353;hsa-miR-4751
ATF5 19 50436338 50436338 TCGA-E2-A15K-06A-11D-A12Q-09 hits on miRNA_primary_transcript;50436321;50436394;hsa-mir-4751
ATF5 19 50436338 50436338 TCGA-E2-A15K-06A-11D-A12Q-09 hits on miRNA;50436330;50436353;hsa-miR-4751
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
F8A1 X 154115715 154115715 TCGA-AN-A0AK-01A-21W-A019-09 hits on miRNA_primary_transcript;154115635;154115733;hsa-mir-1184-1
FOXF2 6 1390570 1390570 TCGA-AO-A03M-01B-11D-A10M-09 hits on miRNA_primary_transcript;1390549;1390646;hsa-mir-6720
FOXF2 6 1390570 1390570 TCGA-AO-A03M-01B-11D-A10M-09 hits on miRNA;1390559;1390580;hsa-miR-6720-3p
IGLV2-8 22 23165351 23165351 TCGA-AO-A0J2-01A-11W-A050-09 hits on miRNA_primary_transcript;23165270;23165365;hsa-mir-650
GL000209.1
GL000209.1
GL000209.1
GL000209.1
GL000209.1
GL000209.1
GL00020

In [4]:
c.rename(columns={0:"Chrom"}, inplace =True)
ms.rename(columns={0:"mStart"}, inplace=True)
me.rename(columns={0:"mEnd"}, inplace=True)
rs.rename(columns={0:"rStart"}, inplace=True)
re.rename(columns={0:"rEnd"}, inplace=True)
t.rename(columns={0:"Type"}, inplace=True)
i.rename(columns={0:"ID"}, inplace=True)
s.rename(columns={0:"Tumor Sample Barcode"}, inplace=True)
final = pd.concat([c,ms,me,rs,re,t,i,s], axis=1)
final = final.reset_index(drop=True)
final

,Chrom,mStart,mEnd,rStart,rEnd,Type,ID,Tumor Sample Barcode
0,19,50436338,50436338,50436321,50436394,miRNA_primary_transcript,hsa-mir-4751,TCGA-E2-A15K-06A-11D-A12Q-09
1,19,50436338,50436338,50436330,50436353,miRNA,hsa-miR-4751,TCGA-E2-A15K-06A-11D-A12Q-09
2,19,50436338,50436338,50436321,50436394,miRNA_primary_transcript,hsa-mir-4751,TCGA-E2-A15K-06A-11D-A12Q-09
3,19,50436338,50436338,50436330,50436353,miRNA,hsa-miR-4751,TCGA-E2-A15K-06A-11D-A12Q-09
4,X,154115715,154115715,154115635,154115733,miRNA_primary_transcript,hsa-mir-1184-1,TCGA-AN-A0AK-01A-21W-A019-09
5,6,1390570,1390570,1390549,1390646,miRNA_primary_transcript,hsa-mir-6720,TCGA-AO-A03M-01B-11D-A10M-09
6,6,1390570,1390570,1390559,1390580,miRNA,hsa-miR-6720-3p,TCGA-AO-A03M-01B-11D-A10M-09
7,22,23165351,23165351,23165270,23165365,miRNA_primary_transcript,hsa-mir-650,TCGA-AO-A0J2-01A-11W-A050-09
8,5,38557622,38557622,38557604,38557663,miRNA_primary_transcript,hsa-mir-3650,TCGA-A7-A3IZ-01A-11D-A20S-09
9,2,177015045,177015045,177015031,177015140,miRNA_primary_transcript,hsa-mir-10b,TCGA-D8-A1XW-01A-11D-A14K-09


In [5]:
final.to_excel("BRCAhits.xls")

In [ ]:
hi = pd.read_table(TABLE2)

chrom2hi = {}
for chr in [str(a) for a in range(1,23)] + ["X", "Y"]:
    chrom2hi[chr] = hi[hi["chrom"] == "chr" + chr]
    
def pos2miRbind(chr, start, end):
    area = chrom2hi[str(chr)]
    oc = []
    for idx, row in area.iterrows():
        if row["chromStart"] <= start <= row["chromEnd"] or \
           row["chromStart"] <= end <= row["chromEnd"] or \
           (start < row["chromStart"] and row["chromEnd"] < end):
            miR = "miR binding site" + ";" + str(row["chromStart"]) + ";" + str(row["chromEnd"]) + ";" + str(row["name"])
            oc.append(miR)
    return oc
        
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    try:
        hits = pos2miR(row["Chromosome"], (row["Start_Position"]), (row["End_Position"]))
    except:
        print row["Chromosome"]
        continue
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_Position"], row["End_Position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_Position"]])
        me = me.append([row["End_Position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

In [ ]:
fh = pd.read_table(TABLE3)
chrom2fh = {}
for chr in [str(a) for a in range(1,23)] + ["X", "Y"]:
    chrom2fh[chr] = fh[fh["chrom"] == "chr" + chr]
    print "chr%s\thas %d  \t miR genes" % (chr, len(chrom2fh[chr]))

def pos2polyA(chr, start, end):
    area = chrom2fh[str(chr)]
    oc = []
    for idx, row in area.iterrows():
        if row["txStart"] <= start <= row["txEnd"] or \
           row["txStart"] <= end <= row["txEnd"] or \
           (start < row["txStart"] and row["txEnd"] < end):
            miR = str(row["name2"]) + ";" + str(row["txStart"]) + ";" + str(row["txEnd"]) + ";" + str(row["name"])
            oc.append(miR)
    return oc
        
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    try:
        hits = pos2miR(row["Chromosome"], (row["Start_Position"]), (row["End_Position"]))
    except:
        print row["Chromosome"]
        continue
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_Position"], row["End_Position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_Position"]])
        me = me.append([row["End_Position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

In [ ]:
c.rename(columns={0:"Chrom"}, inplace =True)
ms.rename(columns={0:"mStart"}, inplace=True)
me.rename(columns={0:"mEnd"}, inplace=True)
rs.rename(columns={0:"rStart"}, inplace=True)
re.rename(columns={0:"rEnd"}, inplace=True)
t.rename(columns={0:"Type"}, inplace=True)
i.rename(columns={0:"ID"}, inplace=True)
s.rename(columns={0:"Tumor Sample Barcode"}, inplace=True)
final = pd.concat([c,ms,me,rs,re,t,i,s], axis=1)
final = final.reset_index(drop=True)
final

In [ ]:
final.to_excel("BRCAhits.xls")

In [ ]:
FILE = "C:/Users/jtso/My Documents/LUSC1.txt"

maf = pd.read_table(FILE, usecols=["Hugo_Symbol", "Chromosome", "Start_Position", "End_Position", "Tumor_Sample_Barcode"])

FILE = "../hw6/hsa.gff3"
TABLE2 = "C:/Users/jtso/Downloads/targetScanS.gz"  # miRNA binding sites
TABLE3 = "C:/Users/jtso/Downloads/wgEncodeGencodePolyaV19 (1).gz"  # polyA sites

df = pd.read_table(FILE, comment="#", usecols=["Chromosome", "Start", "End", "Type", "miR Name"])

chr2df = {}
for chr in [str(i) for i in range(1,23)] + ["X", "Y"]:
    chr2df[chr] = df[df["Chromosome"] == "chr" + chr]
    
def pos2miR(chr, start, end):
    chunk = chr2df[str(chr)]  # shrink the search space to the given chr
    out = []
    for idx, row in chunk.iterrows():
        if row["Start"] <= start <= row["End"] or \
           row["Start"] <=  end  <= row["End"] or \
           (start < row["Start"] and row["End"] < end):
            # get the name (either hsa-mir-### or hsa-miR-###)
            miR = str(row["Type"]) + ";" + str(row["Start"]) + ";" + str(row["End"]) + ";" + row["miR Name"].split("Name=")[1].split(";")[0]
            out.append(miR)
    return out

c = pd.DataFrame()
ms = pd.DataFrame()
me = pd.DataFrame()
rs = pd.DataFrame()
re = pd.DataFrame()
t = pd.DataFrame()
i = pd.DataFrame()
s = pd.DataFrame()
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    try:
        hits = pos2miR(row["Chromosome"], (row["Start_Position"]), (row["End_Position"]))
    except:
        print row["Chromosome"]
        continue
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_Position"], row["End_Position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_Position"]])
        me = me.append([row["End_Position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

In [ ]:
hi = pd.read_table(TABLE2)

chrom2hi = {}
for chr in [str(a) for a in range(1,23)] + ["X", "Y"]:
    chrom2hi[chr] = hi[hi["chrom"] == "chr" + chr]
    
def pos2miRbind(chr, start, end):
    area = chrom2hi[str(chr)]
    oc = []
    for idx, row in area.iterrows():
        if row["chromStart"] <= start <= row["chromEnd"] or \
           row["chromStart"] <= end <= row["chromEnd"] or \
           (start < row["chromStart"] and row["chromEnd"] < end):
            miR = "miR binding site" + ";" + str(row["chromStart"]) + ";" + str(row["chromEnd"]) + ";" + str(row["name"])
            oc.append(miR)
    return oc
        
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    try:
        hits = pos2miR(row["Chromosome"], (row["Start_Position"]), (row["End_Position"]))
    except:
        print row["Chromosome"]
        continue
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_Position"], row["End_Position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_Position"]])
        me = me.append([row["End_Position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

In [ ]:
fh = pd.read_table(TABLE3)
chrom2fh = {}
for chr in [str(a) for a in range(1,23)] + ["X", "Y"]:
    chrom2fh[chr] = fh[fh["chrom"] == "chr" + chr]
    print "chr%s\thas %d  \t miR genes" % (chr, len(chrom2fh[chr]))

def pos2polyA(chr, start, end):
    area = chrom2fh[str(chr)]
    oc = []
    for idx, row in area.iterrows():
        if row["txStart"] <= start <= row["txEnd"] or \
           row["txStart"] <= end <= row["txEnd"] or \
           (start < row["txStart"] and row["txEnd"] < end):
            miR = str(row["name2"]) + ";" + str(row["txStart"]) + ";" + str(row["txEnd"]) + ";" + str(row["name"])
            oc.append(miR)
    return oc
        
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    try:
        hits = pos2miR(row["Chromosome"], (row["Start_Position"]), (row["End_Position"]))
    except:
        print row["Chromosome"]
        continue
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_Position"], row["End_Position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_Position"]])
        me = me.append([row["End_Position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

In [ ]:
c.rename(columns={0:"Chrom"}, inplace =True)
ms.rename(columns={0:"mStart"}, inplace=True)
me.rename(columns={0:"mEnd"}, inplace=True)
rs.rename(columns={0:"rStart"}, inplace=True)
re.rename(columns={0:"rEnd"}, inplace=True)
t.rename(columns={0:"Type"}, inplace=True)
i.rename(columns={0:"ID"}, inplace=True)
s.rename(columns={0:"Tumor Sample Barcode"}, inplace=True)
final = pd.concat([c,ms,me,rs,re,t,i,s], axis=1)
final = final.reset_index(drop=True)
final

In [ ]:
final.to_excel("LUSChits.xls")